In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import random
import csv
import json
import pandas as pd
print("Libraries Imported")

Libraries Imported


In [2]:
# Spoofer code
try:

    import sys
    import os

    from fp.fp import FreeProxy
    from fake_useragent import UserAgent
    from bs4 import BeautifulSoup
    from selenium import webdriver
    from selenium.webdriver import Chrome
    from selenium.webdriver.common.keys import Keys
    from selenium.webdriver.common.by import By
    from selenium.webdriver.chrome.options import Options
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.common.action_chains import ActionChains
    from selenium.webdriver.support import expected_conditions as EC
    from selenium.common.exceptions import TimeoutException
    import time
    print('all module are loaded ')

except Exception as e:

    print("Error ->>>: {} ".format(e))


class Spoofer(object):

    def __init__(self, country_id=['US', 'HK'], rand=True, anonym=True):
        self.country_id = country_id
        self.rand = rand
        self.anonym = anonym
        self.userAgent, self.ip = self.get()

    def get(self):
        ua = UserAgent()
        proxy = FreeProxy(country_id=self.country_id, rand=self.rand, anonym=self.anonym).get()
        ip = proxy.split("://")[1]
        return ua.random, ip


class DriverOptions(object):

    def __init__(self):

        self.options = Options()
        self.options.add_argument('--no-sandbox')
        self.options.add_argument('--start-maximized')
        self.options.add_argument('--start-fullscreen')
        self.options.add_argument('--single-process')
        self.options.add_argument('--disable-dev-shm-usage')
        self.options.add_argument("--incognito")
        self.options.add_argument('--disable-blink-features=AutomationControlled')
        self.options.add_argument('--disable-blink-features=AutomationControlled')
        self.options.add_experimental_option('useAutomationExtension', False)
        self.options.add_experimental_option("excludeSwitches", ["enable-automation"])
        self.options.add_argument("disable-infobars")

        self.helperSpoofer = Spoofer()

        self.options.add_argument('user-agent={}'.format(self.helperSpoofer.userAgent))
        self.options.add_argument('--proxy-server=%s' % self.helperSpoofer.ip)


class WebDriver(DriverOptions):

    def __init__(self, path=''):
        DriverOptions.__init__(self)
        self.driver_instance = self.get_driver()

    def get_driver(self):

        print("""
        IP:{}
        UserAgent: {}
        """.format(self.helperSpoofer.ip, self.helperSpoofer.userAgent))

        PROXY = self.helperSpoofer.ip
        webdriver.DesiredCapabilities.CHROME['proxy'] = {
            "httpProxy":PROXY,
            "ftpProxy":PROXY,
            "sslProxy":PROXY,
            "noProxy":None,
            "proxyType":"MANUAL",
            "autodetect":False
        }
        webdriver.DesiredCapabilities.CHROME['acceptSslCerts'] = True

        path = os.path.join(os.getcwd(), '/chromedriver.exe')

        driver = webdriver.Chrome()
        driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
        driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
            "source":
                "const newProto = navigator.__proto__;"
                "delete newProto.webdriver;"
                "navigator.__proto__ = newProto;"
        })

        return driver

all module are loaded 


In [3]:
def create_csv(fields, rows, dirname, filename):
    with open(dirname+filename, 'w', encoding='UTF8', newline='') as csvfile: 
        # creating a csv writer object
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(fields) 
        csvwriter.writerows(rows)
    print(f"Saved csv with name {filename}")

In [8]:
# Create anti bot detection web driver

while True:
    try:
        driver = WebDriver()
        break
    except: # If Driver creation fails try again
        print("Driver creation failed.....Trying again")
        continue
print("Driver Successfully created")

Driver creation failed.....Trying again

        IP:20.47.108.204:8888
        UserAgent: Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/27.0.1453.93 Safari/537.36
        
Driver Successfully created


In [16]:
# Create Driver instance and open the webpage through selenium

driverinstance = driver.driver_instance
url = "https://www.expedia.com/Hotel-Search?adults=2&d1=2022-04-13&d2=2022-04-14&destination=Jaipur%2C%20Rajasthan%2C%20India&directFlights=false&endDate=2022-04-14&hotels-destination=Jaipur&hotels-destination=Jaisalmer&latLong=26.891643000499514%2C75.74652282695956&localDateFormat=M%2Fd%2Fyyyy&partialStay=false&regionId=1669&semdtl=&sort=RECOMMENDED&startDate=2022-04-13&theme=&useRewards=false&userIntent="
driverinstance.get(url)

# Click on the "SHOW MORE" button 20 times to load enough hotels 
for i in range(8):
    sleep_time = 5 + 5*random.random()
    time.sleep(sleep_time)
    driverinstance.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    try:
        button = driverinstance.find_element_by_xpath('//*[@class="uitk-button uitk-button-medium uitk-button-secondary"]')
        button.click()
    except:
        print("Button not Found, ending search")
        break

# Create BeautifulSoup instance and search for each hotel on the page to get it's link

soup = str(BeautifulSoup(driverinstance.page_source, 'html.parser'))
soup = BeautifulSoup(soup, 'lxml')


In [17]:

ol = soup.find('ol' , class_  = 'results-list no-bullet').find_all('li', attrs={"data-stid":"property-listing"})

hotel_url_li = []
hotel_name_li = []
hotel_price_li = []

for li in ol:
    hotel_url = li.find('a', class_='listing__link uitk-card-link')
    hotel_name = li.find('h3' , class_='uitk-heading-5 is-visually-hidden')
    # hotel_price_div = li.find('div', class_ = "uitk-text uitk-type-600 uitk-type-bold uitk-text-emphasis-theme")
    try:
        hotel_price = li.find('div', class_ = "uitk-text uitk-type-600 uitk-type-bold uitk-text-emphasis-theme").text
    except:
        hotel_price = ""
    hotel_url_li.append(hotel_url)
    hotel_name_li.append(hotel_name)
    hotel_price_li.append(hotel_price)

In [18]:
print(len(ol))
print(hotel_price_li)

300
['$84', '$112', '$91', '$83', '$381', '$51', '$64', '$40', '$94', '$160', '$153', '$94', '$99', '$93', '$146', '$51', '$113', '$39', '$339', '$27', '$96', '$26', '$30', '$33', '$56', '$40', '$84', '$112', '$80', '$99', '$47', '$48', '$91', '$60', '$51', '$96', '$107', '$19', '$51', '$66', '$28', '$62', '$86', '$107', '$19', '$32', '$45', '$29', '$94', '$24', '$23', '$55', '$63', '$23', '$24', '$24', '$62', '$23', '$187', '$669', '$24', '$91', '$45', '$162', '$42', '$28', '$42', '$40', '$15', '$19', '$92', '$61', '$56', '$128', '$33', '$31', '$126', '$28', '$25', '$26', '$22', '$334', '$31', '$75', '$24', '$84', '$27', '$19', '$27', '$38', '$45', '$23', '$28', '$41', '$27', '$35', '$37', '$21', '$16', '$27', '$16', '$55', '$30', '$80', '$63', '$47', '$31', '$13', '$16', '$94', '$67', '$82', '$120', '$47', '$24', '$12', '$35', '$37', '$20', '$85', '$12', '$24', '$11', '$75', '$53', '$62', '$16', '$36', '$29', '$40', '$29', '$13', '$112', '$47', '$29', '$40', '$14', '$20', '$114', '$2

In [19]:
hotel_url_list = []
hotel_info = []

hotel_length = len(hotel_url_li)
print(f"Number of hotels found: {hotel_length}\n")
for i in range(hotel_length):
    hotel_url_list.append("https://www.expedia.com"+hotel_url_li[i].attrs['href'])
    ls = [i+1, hotel_name_li[i].text, hotel_price_li[i][1:]]
    hotel_info.append(ls)

Number of hotels found: 300



In [20]:
# Create CSV for name and price of the hotels

fields = ['hotel_id', 'hotel_name', 'hotel_price']
create_csv(fields, hotel_info, "Project-Dataset/hotel urls/", "Jaipur_hotels.csv")

Saved csv with name Jaipur_hotels.csv


In [49]:
df = pd.read_csv('Project-Dataset/Jodhpur_hotel_info.csv')
df.replace(to_replace='Jaisalmer', value = 'Jodhpur')
print(df['city'])

0      Jaisalmer
1      Jaisalmer
2      Jaisalmer
3      Jaisalmer
4      Jaisalmer
         ...    
148    Jaisalmer
149    Jaisalmer
150    Jaisalmer
151    Jaisalmer
152    Jaisalmer
Name: city, Length: 153, dtype: object


In [21]:
# Save all the links in a text file

with open("Project-Dataset/hotel urls/Jaipur_hotel_urls.txt", 'w') as output:
    for row in hotel_url_list:
        output.write(str(row) + '\n')